# Agents for Amazon Bedrock - Associate Knowledge Base to Agent

This notebook provides sample code for associating a Knowledge Base for Amazon Bedrock to an existent Agent for Amazon Bedrock that has an Action Group attached to it.

### Use Case
We will update our restaurant assistant to allow customers to also ask questions about the restaurant menus. The architecture looks as following:

![Agent Architecture](./images/agent-architecture.png)

### Notebook Walk-through

In this notebook we will:
- Retrieve the saved variables from the previous notebook
- Create a Knowledge Base and its pre-requirements (including OpenSearch Servelless Collection and Indexes)
- Synchronize Knowledge Base with documents containing restaurant menus
- Update Agent IAM role to allow for Knowledge Base access
- Associate Knowledge Base with Restaurant Agent
- Test Agent invocation with Knowledge Base access


### Next Steps: 
In the next lab, we will test the agent invocation with Action Group and Knowledge Base requests as well as provide extra information to the agent using Prompt attributes

### Pre-requisites

Before starting this lab, we need to load the variables that we stored in the previous notebook.

In [2]:
%store -r

In [3]:
agent_id

'O096M3WGHH'

Let's now import the necessary libraries and initiate the required boto3 clients

In [4]:
from knowledge_base import KnowledgeBasesForAmazonBedrock
from agent import invoke_agent_helper
import boto3
import os
import time
import json

In [5]:
iam_client = boto3.client('iam')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
s3_client = boto3.client('s3')

### Setting up Knowledge Base Information

We will now set the variables that define our Knowledge Base:

- **knowledge_base_name**: provides the name of the Knowledge Base to be created, in this case `booking-agent-kb`
- **knowledge_base_description**: the description of the knowledge base used to display the agents list on the console. This description is **not** part of the agent's prompts
- **bucket_name**: name of the bucket containing the Knowledge Base documents

In [6]:
knowledge_base_name = f'{agent_name}-kb'
knowledge_base_description = "Knowledge Base containing the restaurant menu's collection"
bucket_name = f'{agent_name}-{suffix}'

### Creating Knowledge Base for Amazon Bedrock

We will now going to create a Knowledge Base for Amazon Bedrock and its requirements including:
- [Amazon OpenSearch Serverless](https://aws.amazon.com/opensearch-service/features/serverless/) for the vector database
- [AWS IAM](https://aws.amazon.com/iam/) roles and permissions
- [Amazon S3](https://aws.amazon.com/s3/) bucket to store the knowledge base documents

To create the knowledge base and its dependencies, we will use the `BedrockKnowledgeBase` support class, available in this folder. It allows you to create a new knowledge base, ingest documents to the knowledge base data source and delete the resources after you are done working with this lab

In [7]:
kb = KnowledgeBasesForAmazonBedrock()
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(knowledge_base_name, knowledge_base_description, bucket_name)

Creating KB booking-agent-kb
Step 1 - Creating or retrieving booking-agent-us-west-2-296546723528 S3 bucket for Knowledge Base documents
Creating bucket booking-agent-us-west-2-296546723528
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_459) and Policies
Step 3 - Creating OSS encryption, network and data access policies
Step 4 - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '312',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Wed, 26 Jun 2024 17:31:21 '
                                                 'GMT',
                                         'x-amzn-requestid': 'dcf4d2c2-55db-4562-ab52-dac6f5029cf2'},
                        'HTTPStatusCode': 200,
                        'RequestId': 'dcf4d2c

[2024-06-26 17:39:26,178] p216 {base.py:258} INFO - PUT https://dhuceg1i2bzduqi3c5d5.us-west-2.aoss.amazonaws.com:443/booking-agent-kb-index-459 [status:200 request:2.794s]



Creating index:
{ 'acknowledged': True,
  'index': 'booking-agent-kb-index-459',
  'shards_acknowledged': True}
Step 6 - Creating Knowledge Base
{'type': 'VECTOR', 'vectorKnowledgeBaseConfiguration': {'embeddingModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v2:0'}}
{ 'createdAt': datetime.datetime(2024, 6, 26, 17, 40, 26, 381112, tzinfo=tzlocal()),
  'description': "Knowledge Base containing the restaurant menu's collection",
  'knowledgeBaseArn': 'arn:aws:bedrock:us-west-2:296546723528:knowledge-base/HC6ST7J7CP',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v2:0'}},
  'knowledgeBaseId': 'HC6ST7J7CP',
  'name': 'booking-agent-kb',
  'roleArn': 'arn:aws:iam::296546723528:role/AmazonBedrockExecutionRoleForKnowledgeBase_459',
  'status': 'CREATING',
  'storageConfiguration': { 'opensearchS

We now upload the knowledge base documents to S3

In [8]:
def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                file_to_upload = os.path.join(root,file)
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload,bucket_name,file)

upload_directory("kb_documents", bucket_name)

uploading file kb_documents/Restaurant_Childrens_Menu.pdf to booking-agent-us-west-2-296546723528
uploading file kb_documents/Restaurant_Dinner_Menu.pdf to booking-agent-us-west-2-296546723528
uploading file kb_documents/Restaurant_week_specials.pdf to booking-agent-us-west-2-296546723528


And ingest the documents to the knowledge base

In [9]:
# ensure that the kb is available
i_status = ['CREATING', 'DELETING', 'UPDATING']
while bedrock_agent_client.get_knowledge_base(knowledgeBaseId=kb_id)['knowledgeBase']['status'] in i_status:
    time.sleep(10)

# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

{ 'dataSourceId': 'A8JYCVPB5B',
  'ingestionJobId': 'M87BLT0MWV',
  'knowledgeBaseId': 'HC6ST7J7CP',
  'startedAt': datetime.datetime(2024, 6, 26, 17, 41, 30, 294104, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 0,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 0},
  'status': 'STARTING',
  'updatedAt': datetime.datetime(2024, 6, 26, 17, 41, 30, 294104, tzinfo=tzlocal())}
{ 'dataSourceId': 'A8JYCVPB5B',
  'ingestionJobId': 'M87BLT0MWV',
  'knowledgeBaseId': 'HC6ST7J7CP',
  'startedAt': datetime.datetime(2024, 6, 26, 17, 41, 30, 294104, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 3,
  

### Testing Knowledge Base

Let's now test that the created knowledge base works as expected. To do so, we first retrieve the knowledge base id

Next we can use the [`RetrieveAndGenerate`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) API from boto3 to retrieve the context for the question from the knowledge base and generate the final response

In [10]:
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": "Which are the 5 mains available in the childrens menu?"
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, agent_foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

The 5 mains available in the children's menu are:

1. Mini Cheeseburgers - Small beef patties topped with cheese, served on mini buns.
2. Fish Sticks - Breaded fish sticks served with tartar sauce.
3. Grilled Cheese Sandwich - Melted cheese between slices of buttered bread, grilled to perfection.
4. Spaghetti with Marinara Sauce - Kid-friendly spaghetti noodles topped with tomato marinara sauce.



### Updating Agent role to allow Knowledge Base Retrieve and Retrieve and Generate queries

Now that our Knowledge Base is working, we will associate it with the agent. To do so, we first need to update the agent role to allow for retrieval from context from our knowledge base

In [11]:
kb_policies_statements = [
    {
        "Sid": "QueryKB",
        "Effect": "Allow",
        "Action": [
            "bedrock:Retrieve",
            "bedrock:RetrieveAndGenerate"
        ],
        "Resource": [
            f"arn:aws:bedrock:{region}:{account_id}:knowledge-base/{kb_id}"
        ]
    }
]
bedrock_agent_kb_policy_statement = {
    "Version": "2012-10-17",
    "Statement": kb_policies_statements
}
bedrock_agent_kb_policy_json = json.dumps(bedrock_agent_kb_policy_statement)
kb_policy_name = f"{agent_name}-kb-{kb_id}"
agent_kb_policy = iam_client.create_policy(
    PolicyName=kb_policy_name,
    PolicyDocument=bedrock_agent_kb_policy_json
)
iam_client.attach_role_policy(
    RoleName=agent_role['Role']['RoleName'],
    PolicyArn=agent_kb_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': 'f60839de-2ba3-4925-a7b5-e22a0ffd8b9e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 26 Jun 2024 17:42:27 GMT',
   'x-amzn-requestid': 'f60839de-2ba3-4925-a7b5-e22a0ffd8b9e',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

#### Associate Knowledge Base with Agent

Finally, we can associate the new knowledge base with the agent using the [`AssociateAgentKnowledgeBase`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/associate_agent_knowledge_base.html) API from boto3

In [12]:
#time.sleep(10)
response = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description='Access the knowledge base when customers ask about the plates in the menu.',
    knowledgeBaseId=kb_id,
    knowledgeBaseState='ENABLED'
)

#### Preparing Agent

after updating our agent, we need to prepare it again to package all its new components

In [13]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)
# Pause to make sure agent is prepared
intermediate_agent_status = ['CREATING', 'PREPARING', 'UPDATING', 'VERSIONING']
while bedrock_agent_client.get_agent(agentId=agent_id)['agent']['agentStatus'] in intermediate_agent_status:
    time.sleep(10)


{'ResponseMetadata': {'RequestId': '48ed6dd0-f145-4aa4-9238-9d9de3ffdac2', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Wed, 26 Jun 2024 17:42:27 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': '48ed6dd0-f145-4aa4-9238-9d9de3ffdac2', 'x-amz-apigw-id': 'Z_HojGnQPHcEmvg=', 'x-amzn-trace-id': 'Root=1-667c5303-7548f5b35095194e6558fc1b'}, 'RetryAttempts': 0}, 'agentId': 'O096M3WGHH', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2024, 6, 26, 17, 42, 27, 278145, tzinfo=tzlocal())}


### Invoking Agent

Now that our Agent has been updated, let's test it again. To do so we will again use the [`invoke_agent`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/invoke_agent.html) function from the boto3 Bedrock runtime client.

We will use the same support function called `invoke_agent_helper` to allow us to invoke the agent with or without trace enabled and with or without session state. We will get into more details about those concepts in the `03_invoke_agent.ipynb` notebook.

Now we can test it by asking a question where the answer is available in the knowledge base documents

In [14]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = "What are the starters in the childrens menu?"
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

Based on the provided menu, the children's menu starters or appetizers include:

1. Chicken nuggets - Crispy chicken nuggets served with ketchup or ranch dressing.
2. Mini cheese quesadillas - Small flour tortillas filled with melted cheese, served with mild salsa.
3. Peanut butter and banana sandwich - Peanut butter and banana slices on whole wheat bread.
4. Veggie pita pockets - Mini whole wheat pita pockets filled with hummus, cucumber, and cherry tomatoes.<sources>2</sources>
CPU times: user 18.3 ms, sys: 707 µs, total: 19 ms
Wall time: 15.9 s


### Next Steps

Before moving to the next notebook, let's store a couple of variables to continue working the the same notebook.

Next we will test our agent

In [15]:
%store kb_id
%store knowledge_base_name
%store knowledge_base_description
%store kb_policy_name
%store bucket_name
%store knowledge_base_name

Stored 'kb_id' (str)
Stored 'knowledge_base_name' (str)
Stored 'knowledge_base_description' (str)
Stored 'kb_policy_name' (str)
Stored 'bucket_name' (str)
Stored 'knowledge_base_name' (str)
